<h1>Clustering</h1>
Il clustering e' utile ai fini di raggruppare un insieme di oggetti, ove ogni singolo gruppo ha oggetti molto simili tra di loro.<br></br>Questa tecnica e' la forma piu' comune di <i>apprendimento non supervisionato</i>, composto da un insieme di oggetti, di cui non si conosce a priori la corretta classificazione.<br></br>
Esiste anche una versione semi-supervisionata del clustering (<i>Semi-Supervised Clustering</i>), in cui una parte ristretta degli esempi a disposizione, possegono dei vincoli di appartenenza ai vari cluster. Abitualmente l'uso di questa informazione aggiuntiva ha come finalita' migliorare la classificazione.
    

<h2>Definizione della tecnica</h2>

Un problema di clustering puo' venire come di seguito definito:
* insieme di esempi X = {$x_1,...x_n$};
* misura di similarita' tra le varie istanze;
* numero desiderato di cluster K (puo' essere mancante).

L'obiettivo e' quello di calcolare una funzione di assegnamento ($\gamma$) tale che, per ogni oggetto in X, assegna questo oggetto a un solo valore di K:
<center>$\gamma$ : X $\rightarrow$ {1,...,K}</center>

La funzione $\gamma$ fa in modo che nessun assegnamento, nei cluster, risulti vuoto; e che non si abbia nessuna relazione di ordine trai  cluster, definiti solo da numeri simbolici. Inoltre definsce l'<i>hard-clustering</i>, perche' caratterizza un assegnamento deterministico.

Data pero' in questo modo, risulta evidente, che la definizione di cluster appare vaga e soggettiva (come si classifica un frutto, per forma o colore? In entrambi i casi il clustering risulta sensato). Tuttavia tale incertezza, viene sciolta, con la definizione della topologia di clusterizzazione e con la scelta dell'algortimo.

- La topologia di clusterizzazione di un problema, e' definibile sulle seguenti decisioni:
<ol> 
    <li> <b>Come rappresentare gli oggetti nel clustering </b></li>
    Lo scopo che si persegue con il clustering, e' quello di dare una rappresentazione in un feature spaces, cercando di rappresentare i dati, in modo da
    rispettare la conoscenza a priori in possesso.<br></br>
    Fondamentale e' stabilire quale similarita' e quale distanza utilizzare. Per fare questo, si puo' decidere di impiegare un kernel, stabilendo pero' quale e' 
    il piu' adatto, sulla base della conoscenza (difatti esistono non solo diversi kernel per diversi oggetti, ma anche diversi kernel per 
    uno stesso oggetto).<br></br>
    Ovviamente scelte differenti, su ognuno di questi campi, comporta la construzioni di cluster diversi.
    <br></br>
    <li> <b>Quanti cluster si va a considerare</b></li>
    Considero il caso di avere cifre monoscritte, allora potrei decidere di usare 10 cluster, uno per ogni cifra, e analizzare come le immagini, delle cifre, si
    distribuiscono al loro interno. Ma potrei anche decidere di non fissare K  a priori, e farmi guidare dai dati; in questo secondo caso mi sarebbe essenziale usare le
    informazioni che ho dai dati, per riuscire a ottenere il numero ottimale di cluster da adoperare.<br></br>
    In entrambe le scelte, e in generale per ogni problema di clusterizzazione, devo definire un numero di cluster che sia non banale; evitando di costruire cluster
    troppo popolati o con solo 1 o 2 esempi, non in grado di darmi alcuna informazione sulla classificazione del dataset.
</ol> 

- Ci sono 2 categorie di algoritmi di clustering:
<ol>
<li> <b>algoritmi di partizionamento</b></li> Che partono da una partizione random che incrementalmente viene migliorata;
    <li> <b>algoritmi genarchici</b></li> Caratterizzati da  approcci bottom-up o agglomerativi, in cui all'inizio ogni esempio rappresenta un cluster, che successivamente vanno ad aggregarsi; top - down o divisivi, in cui tutti gli esempi rappresentano un unico cluster, successivamente diviso in base alla similarita'.

<h2>Funzione obiettivo e assegnamento</h2>

Il problema del clustering si puo' formulare con una funzione obiettivo, che si presenta per cio' come un problema di ricerca/ottimizzazione, con lo scopo di minimizzazione dell'errore, di un dato di trovarsi assegnato a un certo cluster. Usualmente si usa la distanza.


Svolgere una ricerca esaustiva su K cluster, significa che su n esempi ognuno va assegnato a uno dei K cluster. Tuttavia K$^n$ appare come una stima eccessiva. Difatti, il numero di assegnamenti possibili deve essere invariante al numero assegnato ai cluster. Cio' significa, che se ho i campioni 10 e 11 e sono entrambi assegnati all'cluster 3, e' la stessa cosa che fossero assegnati al cluster 2, l'importante e' solo mantenere l'immutabilita' delle relazioni e non la numerazione. Dunque si parla di K$^n$/K! possibilita' di assegnazione; che tuttavia rimane un numero molto grande, all'aumentare di n. Ecco che l'idea di una ricerca esaustiva per fare classificazione non viene usata; ecco che, come accennavo nel paragrafo precedente, per questo vengono impiegati algoritmi di partizionamento o gerarchici, che usano un approccio greedy per raffinare la scelta del cluster.


<h2>Dataset `COVID-19 in Italiy`</h2>

Di seguito implento dei cluster, con l'uso di algoritmi di partizionamento e genarchici. Il caso studio che ho deciso di utilizzare, sono i dati dei contagi delle regioni italiane, dall'inizio della pandemia COVID-19. Sono interessata al dataset sulle province; i dati sono aggiornati a dicembre 2020.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        df = os.path.join(dirname, filename)
        print(df)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<h3>Creazione del dataset</h3>

In [ ]:
covid_ita_df = pd.read_csv(df, usecols=["SNo", "Date", "RegionCode","TotalPositiveCases"], encoding='latin-1')
covid_ita_df.head(10)

In [ ]:
X = covid_ita_df.drop("Date",axis=1).values # data set
y = covid_ita_df["Date"].values # features

print("Dimensione di X: ", X.shape)
print("Dimensione di y: ", y.shape)


<h3>Algoritmi di partizionamento</h3>

* <h3>K-MEANS</h3>

K-MEANS e' l'algoritmo piu' usato, e parte dall'assunzione che gli esempi sono vettori a valori reali.

Per ogni cluster, la funzione obiettivo cerca di minimizzare la media della distanza tra gli esempi e il centro del cluster
<center>$\mu(c) = \frac{1}{|C|} \sum_{x \in C}x$</center>
ove C e' il centroide.

I punti principali dell'algoritmo di K-MEANS sono i seguenti:
1. *inizializzazione*: vengono generati K punti nello spazio delle istanze, e questi non sono altro che i centroidi dei cluster. Nella versione piu' semplice, dell'algoritmo, (random) vengono presi K esempi inizializzati a caso;
2. *assegnazione*: a ogni esempio viene assengato il cluster del centroide piu' vicino;
3. *ricalcolo*: viene ricalcolata la posizione dei K centroidi, come la media di ogni esempio compreso il vecchio centroide;
4. *loop*: i passi (2) e (3) vengono ripetuti, fino alla stabilizzazione dei centroidi.

Per prima cosa ho dovuto predisporre i dati del dataset, in modo che fosse possibile applicare l'algoritmo di K-MEANS. Indispensabile, a tale fine, e' stato il rimpiazzo del campo `Date`, espresso  in stringa, con un valore numerico rappresentante il mese della rilevazione del campione.

In [ ]:
i = 0
for data in y:
    if '-01-' in data:
        y[i] = "1"       
    elif '-02-' in data:
        y[i] = "2"
    elif '-03-' in data:
        y[i] = "3"
    elif '-04-' in data:
        y[i] = "4"
    elif '-05-' in data:
        y[i] = "5"
    elif '-06-' in data:
        y[i] = "6"
    elif '-07-' in data:
        y[i] = "7"
    elif '-08-' in data:
        y[i] = "8"
    elif '-09-' in data:
        y[i] = "9"
    elif '-10-' in data:
        y[i] = "10"
    elif '-11-' in data:
        y[i] = "11"
    else:
        y[i] = "12"
    i = i + 1

Dopodiche' ho svolto K-Means clustering.

Il mio obiettivo e' stato svolgere date clustering, sul numero di positivi per regione, in modo da individuare un cluster per ogni mese, in cui c'e' stata rilevazione di dati.

Per fare questo, nel metodo offerto da sklearn per K-MEANS, ho deciso:
- di settare il numero di cluster a 11 (`n_clusters`);
- di usare un numero casuale di osservazioni da utilizzare per individuare i centroidi (`init`), prediligendo per motivi di efficienza cosi random a k- means++;
- il numero di volte in cui l'algoritmo viene eseguito (`n_init`),valutando come fosse meglio scegliere un valore piccolo (1) anziche' piu' grande. Tale scelta e' stata vincolta, ancora una volta, da motivi di efficienza. Molto probabilmente la funzione obiettivo ha molti minimi locali, che fanno si che partendo da punti di patenza diversi, si arrivi anche a cluster finali molto diversi; degradando l'accuratezza dei cluster individuati.

In [ ]:
from sklearn.cluster import KMeans


k_means = KMeans(n_clusters=11, init='random', n_init=1) 
k_means.fit(X)

Il campo `inertia` rappresenta la somma delle distanze al quadrato dei campioni dal centro del cluster più vicino, che non e' altro che la funzione obiettivo.

In [ ]:
# funzione obiettivo
k_means.inertia_

Poi ho dovuto occuparmi della rappresentazione dei cluster veri e propri. Per fare questo ho ritenuto, fosse utile, assegnare a ogni mese un colore differente, in modo da rendere piu' evidente i limiti dell'applicazione di K-MEANS, sul set di dati `COVID-19 in Italy`. Difatti, come si vede dalla figura sottostante, i centroidi dei cluster appaiono maggiormente dove si ha un numero di casi a incidenza ridotta (in prossimita' dello 0), con valore di regione 11,12,13. Questo perche', il COVID-19, ha avuto una prima maggiore incidenza nelle regioni del Nord (classificate con valori da 1 a 8), e le restanti sono state soggette a un susseguirsi di casi positivi in prossimita' dello 0, almeno fino al mese di Marzo inoltrato. Questo fa si che lo 0 sia la numerazione piu' diffusa e incidente all'interno dei dati.

In [ ]:
import pylab as plt

plt.rcParams['figure.figsize'] = [10, 10]

# la creazione dell'arraydei colori, con riferimento alla classe di appartenenza
y_color = [];
pos = 0
for c in y:
    if c=="1":
        y_color.append("white")
    elif c=="2":
        y_color.append("silver")
    elif c=="3":
        y_color.append("black")
    elif c=="4":
        y_color.append("purple")
    elif c=="5":
        y_color.append("yellow")
    elif c=="6":
        y_color.append("gray")
    elif c=="7":
        y_color.append("lime")
    elif c=="8":
        y_color.append("navy")
    elif c=="9":
        y_color.append("aqua")
    elif c=="10":
        y_color.append("green")
    elif c=="11":
        y_color.append("fuchsia")
    elif c=="12":
        y_color.append("orange")
        
    pos = pos + 1


# 0 id
# 1 regione
# 2 contagi
f0, f1 = 1,2
plt.scatter(X[:,f0], X[:,f1], marker = 'o', c = y_color)
plt.scatter(k_means.cluster_centers_[:,f0], k_means.cluster_centers_[:,f1], marker = '^', c = "red")
plt.show()


In conclusione, alla discussione dell'immagine sopra, il gruppo di centroidi in posizione centrale, sta cercando di rappresentare i mesi di febbraio e quelli estivi, che hanno avuto un numero basso di casi, ove pero' la sparsita' dello 0, nelle regioni del Sud, incide male sulla formazione dei cluster. Invece i primi centroidi fanno riferimento, alle regioni del Nord, che hanno avuto un'incidenza maggiore di casi sia nei primi mesi della pandemia che durante l'ultimo trimestre dell'anno.

I centroidi rispecchiano poco la classificazione dei dati, in base ai mesi, proprio per:
- la presenza di un numero elevato di casi di positivita' a 0 per molte regioni;
- i numeri di positivi hanno un andamento a picchi, che cresce molto rapidamente.

Tali caratteristiche incidono negativamente sull'accuratezza della classificazione con l'uso dei cluster, come appare evidente anche dal calcolo di RandIndex che ho posto di seguito.

RandIndex e' un criterio esterno che permette di valutare la qualita' della classificazione in rapporto al "ground truth", che in questo caso, e' rappresentato da `y`. Lo scopo di questo indice e' valutare per ogni coppia di esempi, se sono stati correttamente distribuiti nel cluster. Nella situazione ottima si vorrebbe che ogni cluster corrispondesse a una classe del ground truth, con relazioni mantenute.\
In sklearn RandIndex e' implementato dal metodo `adjusted_rand_score`.

In [ ]:
from sklearn import metrics

metrics.adjusted_rand_score(k_means.labels_, y)# RandIndex

<h3>Algoritmi gerarchici</h3>
Questa classe di algoritmi e' utile quando il numero di cluster K non e' dato a priori.

Fondamentale per fare clustering, con algoritmi gerarchici, e' la costruzione del dendogramma.

* <h3>Costruzione del dendogramma</h3>
Costruire un dendogramma significa costruire una tassonomia dagli insiemi di esempi contenuti nel dataset.

Ci sono 4 diverse tecniche che permettono la costruzione di un dendogramma:
* *single-link*:  in questo caso la similarita' tra due classi, e' intesa come la similarita' tra gli esempi piu' simili. Essendo che l'obiettivo e' fondere i cluster con una similarita' simile, questi sono quelli con una distanza tra gli esempi, di cluster diversi, inferiore.
* *complete-link*: in questo caso la misura di similarita' coincide con i cluster piu' dissimili. Dunque vengono fusi i cluster con una distanza tra gli esempi, di cluser diversi, maggiore.
* *average-link*: questo rappresenta una soluzione intermedia a "single-link" e "complete-link". Per tutte le coppie all'interno di due cluster la similarita' e' la similarita' media tra gli esempi dei due cluster.
* *centroid*: in questo caso, per ogni cluster, viene calcolato un centroide, che rappresenta la media, e vengono uniti a due a due i cluster che hanno i centroidi piu' simili.

Ognuna di queste tecniche presenta dei vantaggi e degli svantaggi, che ho potuto testare anche all'interno del dataset `Covid-19 in Italy`. 

Per prima cosa ho proceduto con la costruzione dei dendogrammi per ognuno delle 4 tecniche; dopodiche' ho proceduto a scegliere, in ognuno di essi, quale fosse il valore di soglia ottimale, che mi avrebbe permesso l'individuazione di 11 componeti connesse (i mesi di rilevazione dei dati covid). In fine, per avere la stima della qualita' dei cluster (le componenti connesse) che ho mappato, ho calcolato come gia' avevo fatto per la tecnica K-MEANS, il criterio RandIndex, con la chiamata al metodo `adjusted_rand_score`. Questo mi ha permesso anche un confronto tra le categorie di algoritmi gerarchici e di partizionamento.

In [ ]:
import scipy.cluster.hierarchy as h

<h4> Single-link, Average-link e Centroid</h4>

In [ ]:
plt.rcParams['figure.figsize'] = [10, 8]

# single-link
import sys
sys.setrecursionlimit(10000)
link_matrix1= h.single(X)
h.dendrogram(link_matrix1)
plt.show()

# average-link
link_matrix2= h.average(X)
h.dendrogram(link_matrix2)
plt.show()

# centroid
link_matrix3= h.centroid(X)
h.dendrogram(link_matrix3)
plt.show()

In [ ]:
# single-link
fcluster1 = h.fcluster(link_matrix1, 2950, criterion = "distance")

# average-link
fcluster2 = h.fcluster(link_matrix2, 13000, criterion = "distance")

# centroid
fcluster3 = h.fcluster(link_matrix3, 12500, criterion = "distance")

# single-link
print("single-link: ", metrics.adjusted_rand_score(fcluster1, y))# RandIndex
# average-link
print("average-link: ", metrics.adjusted_rand_score(fcluster2, y))# RandIndex
# centroid
print("centroid: ", metrics.adjusted_rand_score(fcluster3, y))# RandIndex

<h4>Complete-link</h4>

In [ ]:
# complete-link
link_matrix= h.complete(X)
print(link_matrix)

I metodi `single`, `complete`, `centroid` e `average` ritornano una linkage matrix, in cui:
- ogni riga rappresenta un merge dell'algoritmo;
- le prime due colonne, indicano rispettivamente gli indici di una coppia che l'algoritmo unisce (quando un cluster e' identificato da un numero superiore rispetto al numero di esempi, contenuti nel dataset, allora e' un cluster che e' gia' stato fuso con un altro);
- la terza colonna, indica la distanza tra i due cluster;
- la quarta colonna, il numero di esempi totali, contenuti all'interno della coppia di cluster in esame.

Nella matrice `link_matix`(dunque con riferimento al metodo `complete`), per esempio, la prima riga indica, usualmente, che vengono fusi i custer 0 e 1; la distanza tra i due e' 1; ed essendo che sia 0 che 1 sono inferiori a 40201, numero totale di esempi del dataset `COVID-19 in Italy`, sono entrambi cluster singleton, e di conseguenza con un numero di esempi complessivo pari a 2. Invece, nell'ultima riga della matrice, usualmente, si hanno cluster con numerazioni superiori al numero di elementi del dataset, come 80398 e 80399 (perche' entrambi gia' fusi in precedenza) ecco che, non essendo piu' composti da un singolo esempio, hanno un numero di elementi superiore a 2.

Osservando le matrici linkage risultanti, posso dire che e' dimostrato il rispetto della monotonia fra le distanza, esclusivamente nei casi average, single-link e complete-link. Tale peculiarita' mi fa escludere centroid come strategia ottimale per la costruzione del dendogramma.

In [ ]:

plt.rcParams['figure.figsize'] = [20, 8]

h.dendrogram(link_matrix)
plt.show()

In [ ]:
fcluster = h.fcluster(link_matrix, 26000, criterion = "distance")
print("complete-link: ", metrics.adjusted_rand_score(fcluster, y))# RandIndex

Ai fini di questo lavoro, io preferisco l'impiego della tecnica `complete-link`. Di norma e' migliore usare la tecnica `average` in quanto non e' sensibile ne' agli outliers (svantaggio della tecnica complete) ne' tende a fare cluster allungati (chaining effect, svantaggio della tecnica single), ne' le distanze della matrice linkage si presentano non monotone (svantaggio della tecnica `centroid`). Tuttavia il dataset in analisi non presenta valori anomali, ma situazioni di crescita-picco-decrescita; e complice il fatto di una qualita' leggermente superiore, ho deciso di prediligere la tecnica complete.

C'e' da dire che, in rapporto alla tecnica K-MEAN, qui la qualita' della classificazione risulta nettamente inferiore. E questo mi fa escludere l'uso proficuo di algoritmi gerarchici per la clusterizzazione del dataset `COVID-19 in Italy`. Tale risultato lo riconduco alla natura stessa del dataset; difatti per ogni regione si crea una curva di contagi che e' difficile categorizzare per mesi. Obiettivo invece della mia clusterizzazione.